# EEG Music Data Preprocessing 

Goal: Classify types of music by EEG features 

3 types of music: White Noise, Classical, Hip-Hop
Lengths of music: 
    White Noise: 5934 recorded samples
    Classical:   6128 recorded samples
    Hip-Hop:     7674 recorded samples
Data collection: 128 samples/s, 32 Channels 

Preprocessing steps:
1. Filter raw data into different frequency bands 
    - Delta (0-4Hz)
    - Theta (4-8Hz)
    - Alpha (8-13Hz)
    - Beta (13-30Hz)
    - Gamma (>30Hz)
2. Split data into 0.1s samples, creating a list of Nsx32 dataframes for each music type
   where Ns is the number of 0.1s samples

Work Done: Steps 1,2

Future work:


In [89]:
#Import necessary libraries

import pandas as pd
import numpy as np
from scipy import signal
import os
import matplotlib.pyplot as plt
import pickle as pkl
# %matplotlib inline 
%matplotlib qt


In [8]:
#Bandpass (BP) filter helper functions

#Creates butterworth BP filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5*fs  # Nyquist frequency, which is half of fs
    low = lowcut/nyq  # Digital butterworth filter cutoffs must be normalized to Nyquist frequency
    high = highcut/nyq
    b, a = signal.butter(order, [low, high], btype="bandpass")
    return b, a

def butter_lowpass(cutFreq,fs,order=5):
    nyq = 0.5*fs
    cutFreq = cutFreq/nyq
    b,a = signal.butter(order,cutFreq,btype="lowpass")
    return b,a 

def butter_highpass(cutFreq,fs,order=5):
    nyq = 0.5*fs
    cutFreq = cutFreq/nyq
    b,a = signal.butter(order,cutFreq,btype="highpass")
    return b,a 

#Applies butterworth BP filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
#     filtered_data = signal.lfilter(b, a, data)
    filtered_data = signal.filtfilt(b,a,data)
    return filtered_data

#Applies butterworth lowpass filter
def butter_lowpass_filter(data, cutFreq, fs, order=5):
    b, a = butter_lowpass(cutFreq,fs,order=5)
    filtered_data = signal.filtfilt(b,a,data)
    return filtered_data

#Applies butterworth lowpass filter
def butter_highpass_filter(data, cutFreq, fs, order=5):
    b, a = butter_highpass(cutFreq,fs,order=5)
    filtered_data = signal.filtfilt(b,a,data)
    return filtered_data

#Applies butterworth BP filter to Pandas dataframe 
def bp_filter_df(df, lowcut, highcut, fs, order):
    rows, cols = df.shape  # Get no. of rows and cols in df
    new_index = range(1, rows+1)
    new_cols = range(1, cols+1)
    # Create new df with same no. of rows and cols
    new_df = pd.DataFrame(index=new_index, columns=new_cols)
    # new_df = new_df.fillna(0) #Fill in 0 for all values
    for i in range(cols):  # Apply bp filter each column (channel) and saves in new_df
        filt_col = butter_bandpass_filter(
            df.iloc[:, i].values, lowcut, highcut, fs, order)
        new_df[i+1] = filt_col
    return new_df

#Applies butterworth lowpass filter to Pandas dataframe 
def lp_filter_df(df, cutFreq, fs, order):
    rows, cols = df.shape  # Get no. of rows and cols in df
    new_index = range(1, rows+1)
    new_cols = range(1, cols+1)
    # Create new df with same no. of rows and cols
    new_df = pd.DataFrame(index=new_index, columns=new_cols)
    # new_df = new_df.fillna(0) #Fill in 0 for all values
    for i in range(cols):  # Apply bp filter each column (channel) and saves in new_df
        filt_col = butter_lowpass_filter(
            df.iloc[:, i].values, cutFreq, fs, order)
        new_df[i+1] = filt_col
    return new_df

#Applies butterworth highpass filter to Pandas dataframe 
def hp_filter_df(df, cutFreq, fs, order):
    rows, cols = df.shape  # Get no. of rows and cols in df
    new_index = range(1, rows+1)
    new_cols = range(1, cols+1)
    # Create new df with same no. of rows and cols
    new_df = pd.DataFrame(index=new_index, columns=new_cols)
    # new_df = new_df.fillna(0) #Fill in 0 for all values
    for i in range(cols):  # Apply bp filter each column (channel) and saves in new_df
        filt_col = butter_highpass_filter(
            df.iloc[:, i].values, cutFreq, fs, order)
        new_df[i+1] = filt_col
    return new_df

In [4]:
#Importing raw data files 

#.csv paths
csvdir = "C:/Users/Wu Di/Documents/EEG-analysis/200108-Readings-csv/"
classical_csvpath = csvdir + "classical-music.csv"
hiphop_csvpath = csvdir + "hip-hop.csv"
whitenoise_csvpath = csvdir + "white-noise.csv"

#Read .csv files
cols_to_use = list(range(3, 35))

#Raw dataframes - each channel is a column
classical_df = pd.read_csv(classical_csvpath, header=None, usecols=cols_to_use)
hiphop_df = pd.read_csv(hiphop_csvpath, header=None, usecols=cols_to_use)
whitenoise_df = pd.read_csv(
    whitenoise_csvpath, header=None, usecols=cols_to_use)

In [29]:
#Selecting filter orders based on Bode Plots, choosing steepest filter possible with no instability 


checkFilters = False

if checkFilters:

    plt.figure(1)
    plt.clf()
    fs = 128
    cutfreqlp = 4.5
    cutfreqhp = 29.5

    plt.figure(1)
    for order in [10,15,20]:
        b, a = butter_lowpass(cutfreqlp, fs, order=order)
        w, h = signal.freqz(b, a, worN=2000)
        plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)

    plt.plot([0, 0.5 * fs], [np.sqrt(0.5), np.sqrt(0.5)],
             '--', label='sqrt(0.5)')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Gain')
    plt.grid(True)
    plt.legend(loc='best')
    plt.title('Low pass for delta band')

    plt.figure(2)
    for order in [50,60,70,80]:
        b, a = butter_highpass(cutfreqhp, fs, order=order)
        w, h = signal.freqz(b, a, worN=2000)
        plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)

    plt.plot([0, 0.5 * fs], [np.sqrt(0.5), np.sqrt(0.5)],
             '--', label='sqrt(0.5)')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Gain')
    plt.grid(True)
    plt.legend(loc='best')
    plt.title('High pass for gamma band')

    plt.figure(3)
    lowcut = 3.5
    highcut = 7.5
    for order in [3,4,5,6,7,8]:
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        w, h = signal.freqz(b, a, worN=2000)
        plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)

    plt.plot([0, 0.5 * fs], [np.sqrt(0.5), np.sqrt(0.5)],
             '--', label='sqrt(0.5)')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Gain')
    plt.grid(True)
    plt.legend(loc='best')
    plt.title('Band pass for Theta band')

    plt.figure(4)
    lowcut = 7.5
    highcut = 13.5
    for order in [8,9,10]:
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        w, h = signal.freqz(b, a, worN=2000)
        plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)

    plt.plot([0, 0.5 * fs], [np.sqrt(0.5), np.sqrt(0.5)],
             '--', label='sqrt(0.5)')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Gain')
    plt.grid(True)
    plt.legend(loc='best')
    plt.title('Band pass for Alpha band')


    plt.figure(5)
    lowcut = 12.5
    highcut = 30.5
    for order in [12,13,14,15,16,17,18]:
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        w, h = signal.freqz(b, a, worN=2000)
        plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)

    plt.plot([0, 0.5 * fs], [np.sqrt(0.5), np.sqrt(0.5)],
             '--', label='sqrt(0.5)')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Gain')
    plt.grid(True)
    plt.legend(loc='best')
    plt.title('Band pass for Beta band')

    plt.show()

Filter orders selected:
Delta - 10 (LP)
Theta - 6 (BP)
Alpha - 8 (BP)
Beta  - 16 (BP)
Gamma - 50 (HP)

In [31]:
#Apply BP filtering to raw dataframes
def filt_freq_bands(df,fs,order):
    delta = lp_filter_df(df, 4.5, fs, 10) 
    theta = bp_filter_df(df, 3.5, 7.5, fs, 6)
    alpha = bp_filter_df(df, 7.5, 13.5, fs, 8)
    beta = bp_filter_df(df, 12.5, 30.5, fs, 16)
    gamma = hp_filter_df(df, 29.5, fs, 50)
    return [delta, theta, alpha, beta, gamma]

fs = 128
order = 6

C_bands_list = filt_freq_bands(classical_df,fs,order)
H_bands_list = filt_freq_bands(hiphop_df,fs,order)
W_bands_list = filt_freq_bands(whitenoise_df,fs,order)

print(len(C_bands_list))

5


In [105]:
#Split filtered dataframes into samples
fs = 128 #sampling freq
sample_t = 0.1 #sample time length in seconds

#Splits a single dataframe into list of equally sized arrays
#Each element in list is nx32 array, where n= sample length 
def split_df(df,fs,sample_t,check=False):
    rows,_ = df.shape #get no. of rows
    sample_len = int(sample_t*fs) #find no. of recorded samples required for each sample time length
    Ns = int(rows/sample_len) #find total no. of samples
    df_cut = df.loc[:Ns*sample_len] #truncate dataframe to exact multiple of sample length
    df_split_list = np.vsplit(df_cut,Ns) #split dataframe row-wise, returns a list
    
    if check:
        print("Total no. of recorded samples: "+str(rows))
        print("Sample length: "+str(sample_len))
        print("Total no. of samples: "+str(Ns))
        print("Length of df_split_list: "+str(len(df_split_list)))
        
        if all(isinstance(x.shape,tuple) for x in df_split_list):
            print("Shape of each element in df_split_list: "+str(df_split_list[0].shape))
        else:
            print("Shapes are wrong.")
            for x in df_split_list:
                print(x.shape)
    return df_split_list,Ns

#Apply split_df() function to list of dataframes, reshape dataframe such that each element is an array 
#for the appropriate sample time length 
def split_bands_list(bands_list,fs,sample_t,check=False,checkSD=False):
    df_list_rFE = [0]*len(bands_list) #dataframes list ready for feature extraction 
    for df_no in range(len(bands_list)):
        df_split_list,Ns = split_df(bands_list[df_no],fs,sample_t,check=checkSD)
        list_of_series = [0]*Ns
        for i in range(len(df_split_list)):
            #New dataframe will have shape Nsx32, each element is a 1xsample_len array 
            new_row = [0]*32 
            #Each df_split_list[i] is a dataframe
            for j in range(len(df_split_list[i].columns)):
                new_row[j] = df_split_list[i].iloc[:,j].values 
            list_of_series[i] = new_row
        df_list_rFE[df_no] = pd.DataFrame(list_of_series)
    if check:
        print("Length of bands_list: "+str(len(bands_list)))
        print("Length of df_list_rFE: "+str(len(df_list_rFE)))
        if (all(isinstance(x.shape,tuple) for x in df_list_rFE)) and (Ns==len(df_list_rFE[0].index)):
            print("Shape of each dataframe in df_list_rFE: "+str(df_list_rFE[0].shape))
    return df_list_rFE
            
C_bands_split_list = split_bands_list(C_bands_list,fs,sample_t,check=True)
H_bands_split_list = split_bands_list(H_bands_list,fs,sample_t,check=True)
W_bands_split_list = split_bands_list(W_bands_list,fs,sample_t,check=True)


Length of bands_list: 5
Length of df_list_rFE: 5
Shape of each dataframe in df_list_rFE: (510, 32)
Length of bands_list: 5
Length of df_list_rFE: 5
Shape of each dataframe in df_list_rFE: (639, 32)
Length of bands_list: 5
Length of df_list_rFE: 5
Shape of each dataframe in df_list_rFE: (494, 32)


In [107]:
#Save lists to .pkl files for feature extraction

cwd = os.getcwd() #current directory
C_filename = cwd+"/pkl/preprocessing/C_bands_0.1s_list.pkl"
H_filename = cwd+"/pkl/preprocessing/H_bands_0.1s_list.pkl"
W_filename = cwd+"/pkl/preprocessing/W_bands_0.1s_list.pkl"

print(C_filename)
print(H_filename)
print(W_filename)

with open(C_filename,"wb") as f:
    pkl.dump(C_bands_split_list,f)
    
with open(H_filename,"wb") as f:
    pkl.dump(H_bands_split_list,f)
    
with open(W_filename,"wb") as f:
    pkl.dump(W_bands_split_list,f)

C:\Users\Wu Di\Desktop\Northwestern!\Senior Year\Spring 2020\ME 499\Code/pkl/preprocessing/C_bands_0.1s_list.pkl
C:\Users\Wu Di\Desktop\Northwestern!\Senior Year\Spring 2020\ME 499\Code/pkl/preprocessing/H_bands_0.1s_list.pkl
C:\Users\Wu Di\Desktop\Northwestern!\Senior Year\Spring 2020\ME 499\Code/pkl/preprocessing/W_bands_0.1s_list.pkl
